# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.7);

In [4]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.1415,0.0,6.91,0,0.448,6.169,6.6,5.7209,3,233
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
3,0.09178,0.0,4.05,0,0.51,6.416,84.1,2.6463,5,296
4,0.1146,20.0,6.96,0,0.464,6.538,58.7,3.9175,3,223
5,24.3938,0.0,18.1,0,0.7,4.652,100.0,1.4672,24,666
6,1.49632,0.0,19.58,0,0.871,5.404,100.0,1.5916,5,403
7,3.32105,0.0,19.58,1,0.871,5.403,100.0,1.3216,5,403
8,13.5222,0.0,18.1,0,0.631,3.863,100.0,1.5106,24,666
9,0.26838,0.0,9.69,0,0.585,5.794,70.6,2.8927,6,391


In [5]:
names(boston)

14-element Array{Symbol,1}:
 :Crim
 :Zn
 :Indus
 :Chas
 :NOx
 :Rm
 :Age
 :Dis
 :Rad
 :Tax
 :PTRatio
 :Black
 :LStat
 :MedV

## Modeling

In [6]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat + MedV), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat + MedV

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error       t value  Pr(>|t|)     Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────────────
(Intercept)  -4.41086e-12  3.2902e-13   -13.4061        <1e-32  -5.05804e-12  -3.76368e-12
Crim          4.7674e-16   1.92333e-15    0.247872      0.8044  -3.30643e-15   4.25991e-15
Zn           -2.2439e-15   8.11153e-16   -2.76631       0.0060  -3.83943e-15  -6.48376e-16
Indus         5.18408e-15  3.64077e-15    1.42389       0.1554  -1.97727e-15   1.23454e-14
Chas          3.62466e-14  5.26271e-14    0.688743      0.4915  -6.72703e-

In [7]:
preds = predict(ols, test)

152-element Array{Union{Missing, Float64},1}:
 25.29999999999993
 33.40000000000001
 23.599999999999753
 24.399999999999867
 10.499999999999787
 19.599999999999593
 13.400000000000194
 23.09999999999852
 18.29999999999992
 18.79999999999989
 32.49999999999957
  8.299999999999839
 37.00000000000012
  ⋮
 31.999999999999833
 15.999999999999748
 13.300000000000527
 15.000000000000234
 19.4999999999999
 12.300000000000054
 21.699999999999907
 21.999999999999886
 20.10000000000001
 24.200000000000152
 10.900000000000418
 18.400000000000098

## Evaluation

In [8]:
GLM.r²(ols)

1.0

In [9]:
GLM.adjr²(ols)

1.0

In [10]:
nrow(test)

152

In [11]:
# RMSE
(sum((preds .- test.MedV) .^ 2) / nrow(test)) .^ (1/2)

2.8995623932085284e-13